<a href="https://colab.research.google.com/github/sanaalshboul/Gender-Voice-Detection/blob/master/Train_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import Google drive
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [7]:
#change the current working directory to the dataset directory
#for dataset downloading: https://www.kaggle.com/liftofff9/voice-gender
import os
os.chdir('/content/drive/My Drive/pygender')
!ls

data			       logs			train_data
Data_preperation.ipynb	       mfcc_features_test.csv	Train_Test.ipynb
ELA_Voice_visualization.ipynb  mfcc_features_train.csv
images			       test_data


In [8]:
#import the required libraries
import pickle
import numpy as np


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,  Conv1D, MaxPooling1D, Flatten,InputLayer
from keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.callbacks import LearningRateScheduler
from keras.callbacks import History
from keras.wrappers.scikit_learn import KerasClassifier


In [9]:
#metho to load training or testing files
def load_data(train_or_test):
  if train_or_test=="train":
    npFeatureFile="data/all_mfcc_train.npy"
    npLabelFile="data/all_label_train.npy"
  elif train_or_test=="test":
    npFeatureFile="data/all_mfcc_test.npy"
    npLabelFile="data/all_label_test.npy"

  X_train=np.load(npFeatureFile)
  Y_train=np.load(npLabelFile)
  
  return X_train, Y_train

In [10]:
#loading the testing and training sets
X_train,Y_train=load_data(train_or_test="train")
X_test,Y_test=load_data(train_or_test="test")

#makes the random numbers predictable
np.random.seed(5)

In [ ]:
#hyperparameters finetuning for XGBClassifier using GridSearchCV algorithm

param_grid={'learning_rate':[.0001,.1,.2,.5], 'n_estimators':[20,50,100,150,300], 'warm_start':[True, False]}
xgb=XGBClassifier()
xgb_cv=GridSearchCV(xgb, param_grid, cv=5)
xgb_cv.fit(X_train, Y_train.ravel())

#model_name="mfcc_model.sav"
#pickle.dump(xgb_cv, open(model_name, 'wb'))

print(xgb_cv.best_params_)
print(xgb_cv.best_score_)

{'learning_rate': 0.0001, 'n_estimators': 20, 'warm_start': True}
0.5


In [ ]:
#training and testing the data set using the best prameters of the XGBClassifier

# Initialize the classifier
xgb=XGBClassifier(learning_rate= 0.0001, n_estimators= 20, warm_start= True)

# Train the classifier
xgb.fit(X_train, Y_train.ravel())

# Make predictions
Y_pred=xgb.predict(X_test)

# Evaluate accuracy
print(accuracy_score(Y_test, Y_pred))
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

0.6843033509700176
              precision    recall  f1-score   support

         0.0       0.73      0.60      0.66       576
         1.0       0.65      0.77      0.71       558

    accuracy                           0.68      1134
   macro avg       0.69      0.69      0.68      1134
weighted avg       0.69      0.68      0.68      1134

[[345 231]
 [127 431]]


In [ ]:
#training and testing the data set using the GradientBoostingClassifier

gb = GradientBoostingClassifier(n_estimators= 600, max_depth=1,max_features=1, min_samples_split=2,
              learning_rate= .001)

gb.fit(X_train, Y_train.ravel())

Y_pred=gb.predict(X_test)

print(accuracy_score(Y_test, Y_pred))
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

0.6331569664902998
              precision    recall  f1-score   support

         0.0       0.71      0.48      0.57       576
         1.0       0.60      0.79      0.68       558

    accuracy                           0.63      1134
   macro avg       0.65      0.64      0.62      1134
weighted avg       0.65      0.63      0.62      1134

[[275 301]
 [115 443]]


In [ ]:
#hyperparameters finetuning for DecisionTreeClassifier using GridSearchCV algorithm

param_grid={"criterion":["gini", "entropy"],  "max_depth": np.arange(1,5),"max_features":["auto", "sqrt", "log2"]}
dtr=DecisionTreeClassifier()

xgb_cv=GridSearchCV(dtr, param_grid, cv=5)
xgb_cv.fit(X_train, Y_train.ravel())

#model_name="mfcc_model.sav"
#pickle.dump(xgb_cv, open(model_name, 'wb'))

print(xgb_cv.best_params_)
print(xgb_cv.best_score_)

{'criterion': 'gini', 'max_depth': 3, 'max_features': 'auto'}
0.9


In [11]:
#training and testing the data set using the best prameters of the DecisionTreeClassifier

dtr=DecisionTreeClassifier( max_depth=3, max_features='log2',criterion="gini",random_state=50).fit(X_train,Y_train)

Y_pred=dtr.predict(X_test)

print(accuracy_score(Y_test, Y_pred))
print(classification_report(Y_test,Y_pred))


0.5731922398589065
              precision    recall  f1-score   support

         0.0       0.57      0.65      0.61       576
         1.0       0.58      0.49      0.53       558

    accuracy                           0.57      1134
   macro avg       0.57      0.57      0.57      1134
weighted avg       0.57      0.57      0.57      1134



In [ ]:
#hyperparameters tuning for a created NN model

epochs=35
learning_rate = 0.001
decay_rate = learning_rate / epochs
momentum = 0.8

#initialize optimizer
sgd = Adam(lr=learning_rate)

def create_model(init_mode='uniform'):
    vector_length=25

    model = Sequential()
    model.add(Dense(256, kernel_initializer=init_mode,input_shape=(vector_length,)))
    model.add(Dense(256,kernel_initializer=init_mode, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, kernel_initializer=init_mode,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128,kernel_initializer=init_mode, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(64, kernel_initializer=init_mode,activation="relu"))
    model.add(Dropout(0.3))
    # one output neuron with sigmoid activation function, 1 means female, 0 means male
    model.add(Dense(1, kernel_initializer=init_mode,activation="sigmoid"))

    # using binary crossentropy as it's male/female classification (binary)
    model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer=sgd)
   
    return model


model_CV = KerasClassifier(build_fn=create_model, verbose=1)

# define the grid search parameters
init_mode = ['huniform' ,'lecun_uniform']
batches = [10,5]
epochs = [10,30,40,60,75,80]

param_grid = dict(epochs=epochs, batch_size=batches,init_mode=init_mode)

grid = GridSearchCV(estimator=model_CV, param_grid=param_grid, n_jobs=-1, cv=3)

grid_result = grid.fit(X_train, Y_train)

#print the best hyperparameters' values of the model
print(f'Best Accuracy for {grid_result.best_score_:.4} using {grid_result.best_params_}')


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/75
1/1 [==============================] - 0s 2ms/step - loss: 0.7857 - accuracy: 0.4000
Epoch 2/75
1/1 [==============================] - 0s 1ms/step - loss: 0.8143 - accuracy: 0.3000
Epoch 3/75
1/1 [==============================] - 0s 998us/step - loss: 0.6511 - accuracy: 0.7000
Epoch 4/75
1/1 [==============================] - 0s 2ms/step - loss: 0.5713 - accuracy: 0.7000
Epoch 5/75
1/1 [==============================] - 0s 1ms/step - loss: 0.6763 - accuracy: 0.6000
Epoch 6/75
1/1 [==============================] - 0s 2ms/step - loss: 0.8869 - accuracy: 0.3000
Epoch 7/75
1/1 [==============================] - 0s 1ms/step - loss: 0.7681 - accuracy: 0.4000
Epoch 8/75
1/1 [==============================] - 0s 2ms/step - loss: 0.7003 - accuracy: 0.4000
Epoch 9/75
1/1 [==============================] - 0s 972us/step - loss: 0.6360 - accuracy: 0.7000
Epoch 10/75
1/1 [==============================] - 0s 2ms/step - loss: 0.7920 - accuracy: 0.3000
Epoch 11/75
1/1 [==================

In [ ]:
#train and test the model using the best parameters
learning_rate = 0.001
momentum = 0.8

sgd = Adam(lr=learning_rate)

def create_model(init_mode='lecun_uniform'):
    vector_length=25
    """5 hidden dense layers from 256 units to 64, not the best model."""
    model = Sequential()
    model.add(Dense(256, kernel_initializer=init_mode,input_shape=(vector_length,)))
    model.add(Dense(256,kernel_initializer=init_mode, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, kernel_initializer=init_mode,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128,kernel_initializer=init_mode, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(64, kernel_initializer=init_mode,activation="relu"))
    model.add(Dropout(0.3))
    # one output neuron with sigmoid activation function, 0 means female, 1 means male
    model.add(Dense(1, kernel_initializer=init_mode,activation="sigmoid"))
    # using binary crossentropy as it's male/female classification (binary)
    model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer=sgd)
    # print summary of the model
   
    return model


model=create_model()

model.fit(X_train,Y_train, batch_size=10, epochs=75, verbose=1)

loss, accuracy = model.evaluate(X_test, Y_test, verbose=1)
print("Loss:", loss)
print("Accuracy: ",accuracy*100)

Epoch 1/75
1/1 [==============================] - 0s 2ms/step - loss: 0.8071 - accuracy: 0.6000
Epoch 2/75
1/1 [==============================] - 0s 4ms/step - loss: 0.8136 - accuracy: 0.5000
Epoch 3/75
1/1 [==============================] - 0s 2ms/step - loss: 0.7592 - accuracy: 0.6000
Epoch 4/75
1/1 [==============================] - 0s 2ms/step - loss: 0.7385 - accuracy: 0.5000
Epoch 5/75
1/1 [==============================] - 0s 2ms/step - loss: 0.6717 - accuracy: 0.6000
Epoch 6/75
1/1 [==============================] - 0s 2ms/step - loss: 0.6191 - accuracy: 0.6000
Epoch 7/75
1/1 [==============================] - 0s 2ms/step - loss: 0.6752 - accuracy: 0.5000
Epoch 8/75
1/1 [==============================] - 0s 2ms/step - loss: 0.7408 - accuracy: 0.6000
Epoch 9/75
1/1 [==============================] - 0s 1ms/step - loss: 0.6982 - accuracy: 0.6000
Epoch 10/75
1/1 [==============================] - 0s 1ms/step - loss: 0.7155 - accuracy: 0.5000
Epoch 11/75
1/1 [======================